In [109]:
import os
import pandas as pd

import plotly.express as px
import plotly.offline as pyo

pyo.init_notebook_mode()

Output hidden; open in https://colab.research.google.com to view.

### Importing Dataset

In [110]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
users_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Tegger/users_data.csv')
snowplow_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Tegger/snowplow_data.csv')

In [112]:
print(users_data.shape)
print(snowplow_data.shape)

(3764, 15)
(23465, 51)


### Understanding Dataset

In [113]:
users_data.head(5)

,Unnamed: 0,user_id,snowplow_pseudo_id,AGEB_key,user_first_visit,traffic_source,device_category,browser,mobile_brand,model_name,country,total_session_duration,total_events,total_page_views,engagement_time_minutes
0,0,e3eabd43-23f3-4781-90ee-5a67e781e7e2,1364061063.1617121653,NaN,2021-03-30,(direct),desktop,Chrome,Google,Chrome,Colombia,2019,12,4,2.42
1,1,e4ccc0ac-71b1-4d7c-b697-f9f76f1084d0,1645287458.1617116080,NaN,2021-03-30,m.facebook.com,mobile,Chrome,Samsung,SM-J710MN,Peru,467,11,4,2.00
2,2,815ef036-f1ee-486e-85a2-facf11df24a1,2096996404.1617125813,NaN,2021-03-30,google,mobile,Chrome,Xiaomi,Redmi 7A,Russia,540,11,2,1.89
3,3,c86127fb-bc84-488a-b961-2abc5213f6fd,575100521.1617133751,NaN,2021-03-30,m.facebook.com,mobile,Android Webview,Samsung,SM-G900H,Ecuador,161,7,2,1.38
4,4,1c101e61-f3cc-4d77-8803-e27aa80f2a59,804160641.1617114854,NaN,NaN,tegger.io,desktop,Chrome,Google,Chrome,Netherlands,81,13,3,0.47


In [114]:
users_data.dtypes

Unnamed: 0                   int64
user_id                     object
snowplow_pseudo_id          object
AGEB_key                    object
user_first_visit            object
traffic_source              object
device_category             object
browser                     object
mobile_brand                object
model_name                  object
country                     object
total_session_duration       int64
total_events                 int64
total_page_views             int64
engagement_time_minutes    float64
dtype: object

In [115]:
snowplow_data.head(5)

,Unnamed: 0,altitude,altitudeAccuracy,bearing,br_colordepth,br_family,br_lang,br_name,br_type,br_viewheight,br_viewwidth,derived_tstamp,derived_tstamp_local,doc_height,doc_width,domain_sessionid,domain_userid,dvce_ismobile,dvce_screenheight,dvce_screenwidth,dvce_type,event_id,event_name,geo_city,geo_country,geo_latitude,geo_longitude,geo_region,geo_region_name,geo_timezone,geo_zipcode,latitudeLongitudeAccuracy,network_userid,osVersion,os_family,os_manufacturer,os_name,page_referrer,page_title,page_url,page_urlpath,pp_max,pp_may,pp_mix,pp_miy,speed,targetUrl,unique_id,user_fingerprint,user_id,pseudo_user_id
0,0,NaN,NaN,NaN,24,Unknown,es-419,Unknown,unknown,753,364,2021-04-15 01:47:28.121,2021-04-14 20:47:28.121,7007,364,1a697c89-1590-49b2-be55-97422246c89a,9466ea61-aa91-46f0-a610-271d712da8fa,0,780,360,Unknown,6fc1bb2b-c4aa-4b16-b061-dc1dea849abf,page_ping,Mexico City,MX,19.3455,-99.1326,CMX,Mexico City,America/Mexico_City,04260,NaN,5229458e-ba53-483b-a7a0-4d9e1943bce9,Other,Unknown,Other,Unknown,https://t.co/,"Prueba de Draize, experimento en Save Ralph - ...",https://culturacolectiva.com/estilo-de-vida/pr...,/estilo-de-vida/prueba-de-draize-experimento-e...,0.0,2872.0,0.0,2320.0,NaN,NaN,E1RCZkfEkX7wwQq6QbLW,3198786281,4888686b-3d8e-4baf-82c1-c2ba0faffa62,1.890160e+08
1,1,NaN,NaN,NaN,24,Unknown,es-419,Unknown,unknown,753,364,2021-04-15 01:47:33.142,2021-04-14 20:47:33.142,7302,364,1a697c89-1590-49b2-be55-97422246c89a,9466ea61-aa91-46f0-a610-271d712da8fa,0,780,360,Unknown,4912d493-f2ce-4f73-b5db-5ba0211ccf78,page_ping,Mexico City,MX,19.3455,-99.1326,CMX,Mexico City,America/Mexico_City,04260,NaN,ebe04960-310e-416c-a909-4b0ce7ba0a70,Other,Unknown,Other,Unknown,https://t.co/,"Prueba de Draize, experimento en Save Ralph - ...",https://culturacolectiva.com/estilo-de-vida/pr...,/estilo-de-vida/prueba-de-draize-experimento-e...,0.0,4011.0,0.0,3451.0,NaN,NaN,LCigDHOvGWg3JUiQmjox,3198786281,4888686b-3d8e-4baf-82c1-c2ba0faffa62,1.890160e+08
2,2,NaN,NaN,NaN,24,Unknown,es-419,Unknown,unknown,753,364,2021-04-15 01:47:24.117,2021-04-14 20:47:24.117,6988,364,1a697c89-1590-49b2-be55-97422246c89a,9466ea61-aa91-46f0-a610-271d712da8fa,0,780,360,Unknown,ad8dd144-092b-4957-bd9c-f580d6a61e16,page_ping,Mexico City,MX,19.3455,-99.1326,CMX,Mexico City,America/Mexico_City,04260,NaN,e6c99964-e37c-471f-88e1-d5abe8ae5c40,Other,Unknown,Other,Unknown,https://t.co/,"Prueba de Draize, experimento en Save Ralph - ...",https://culturacolectiva.com/estilo-de-vida/pr...,/estilo-de-vida/prueba-de-draize-experimento-e...,0.0,1953.0,0.0,1941.0,NaN,NaN,DWIcGw2vEUS2BAHF0MZr,3198786281,4888686b-3d8e-4baf-82c1-c2ba0faffa62,1.890160e+08
3,3,NaN,NaN,NaN,24,Unknown,ko,Unknown,unknown,969,1920,2021-04-15 01:28:40.430,2021-04-15 10:28:40.430,5119,1903,115c4d02-ce53-4d42-8c38-10cc8f1a6c6b,9ddc4239-04ed-4ce6-8029-630a1d4f9ed0,0,1080,1920,Unknown,60daf76e-c1eb-47d4-8037-2e75f44ea101,page_ping,NaN,KR,37.5112,126.9741,NaN,NaN,Asia/Seoul,NaN,NaN,704b1b6f-5427-4f78-b0fd-54aecb695277,Other,Unknown,Other,Unknown,https://news.culturacolectiva.com/,Un sitio de noticias que inspiran | CC News,https://news.culturacolectiva.com/,/,0.0,0.0,0.0,0.0,NaN,NaN,wssRvnBwMABawapJcxXJ,2955001579,b15477c0-41cc-483c-9e3c-4a439ce2df34,3.148460e+08
4,4,NaN,NaN,NaN,24,Unknown,es-419,Unknown,unknown,753,364,2021-04-15 01:47:31.123,2021-04-14 20:47:31.123,7007,364,1a697c89-1590-49b2-be55-97422246c89a,9466ea61-aa91-46f0-a610-271d712da8fa,0,780,360,Unknown,df3039da-e102-4971-9769-7b76b2ac4bee,page_ping,Mexico City,MX,19.3455,-99.1326,CMX,Mexico City,America/Mexico_City,04260,NaN,a38dd5dd-e26d-4c07-87b0-33cd851ece6e,Other,Unknown,Other,Unknown,https://t.co/,"Prueba de Draize, experimento en Save Ralph - ...",https://culturacolectiva.com/estilo-de-vida/pr...,/estilo-de-vida/prueba-de-draize-experimento-e...,0.0,3060.0,0.0,2872.0,NaN,NaN,gRLDeoXOb30dBKE8EWPs,3198786281,4888686b-3d8e-4baf-82c1-c2ba0faffa62,1.890160e+08


In [116]:
snowplow_data.dtypes

Unnamed: 0                     int64
altitude                     float64
altitudeAccuracy             float64
bearing                      float64
br_colordepth                  int64
br_family                     object
br_lang                       object
br_name                       object
br_type                       object
br_viewheight                  int64
br_viewwidth                   int64
derived_tstamp                object
derived_tstamp_local          object
doc_height                     int64
doc_width                      int64
domain_sessionid              object
domain_userid                 object
dvce_ismobile                  int64
dvce_screenheight              int64
dvce_screenwidth               int64
dvce_type                     object
event_id                      object
event_name                    object
geo_city                      object
geo_country                   object
geo_latitude                 float64
geo_longitude                float64
g

In [117]:
## chosse rows which matches the users_id in the list

# users_data.query('user_id in @users_id')
# users_data[users_data['user_id'].isin(users_id)]


# creating a list of User IDs

users_id =users_data['user_id']

print("Before picking out unique values")
print(users_id.shape)
print(users_id.value_counts())

print("\nAfter picking our unique values")
users_id = users_id.unique()
print(users_id.shape)

Before picking out unique values
(3764,)
2be3c4b4-081c-4721-9469-ae2d0a6af473    544
be9335e4-64e4-483b-85db-9eda502802fc    132
91e978ea-78e6-4bfe-96db-2d8b56f87c78    109
1ad495ad-2e1b-4a37-ae3a-9f1dd8fe2931     96
1c101e61-f3cc-4d77-8803-e27aa80f2a59     96
                                       ... 
18c05087-2f95-4639-b354-4b896cb55e7a      1
831edb2a-f16c-4414-9cef-10876f0fc339      1
80267c71-54e1-4e1f-8035-b1d064cee25e      1
fd71592b-47d9-4e0c-908a-79a947162634      1
d93275fb-097f-4d65-ae18-de7321ca9d82      1
Name: user_id, Length: 859, dtype: int64

After picking our unique values
(859,)


There are 859 unique ids who have total sessions time

In [118]:
snowplow_users_id =snowplow_data['user_id'].unique()
print(snowplow_users_id.shape)

(32,)


There are only 32 unique ids in snowplow_data. This mean only for 32 users in users_data we have addtional data in snowplow_data.

## Q1. Is there a correlation between screen size and session duration?

### Determining the measure of screen size: 'device_category' or 'dvce_screenheight', and 'dvce_screenwidth'.

In [119]:
# for random user 1
print(users_data[users_data['user_id']=='2be3c4b4-081c-4721-9469-ae2d0a6af473']['device_category'].value_counts())
print(snowplow_data[snowplow_data['user_id']=='2be3c4b4-081c-4721-9469-ae2d0a6af473']['dvce_screenheight'].value_counts())

desktop    425
mobile      78
tablet      41
Name: device_category, dtype: int64
480    1993
540     555
Name: dvce_screenheight, dtype: int64


In [120]:
# for random user 2
print(users_data[users_data['user_id']=='1c101e61-f3cc-4d77-8803-e27aa80f2a59']['device_category'].value_counts())
print(snowplow_data[snowplow_data['user_id']=='c101e61-f3cc-4d77-8803-e27aa80f2a59']['dvce_screenheight'].value_counts())

desktop    96
Name: device_category, dtype: int64
Series([], Name: dvce_screenheight, dtype: int64)


Not all user ids have device screen height and screen width. So, taking device category seems a better choice for measuring screen size

### Finding whether there exist correlation between 'device_category' and 'total_session_duration'


In [121]:
df1 = users_data.filter(['user_id','device_category','total_session_duration'], axis=1)
print(df1.shape)

(3764, 3)


In [122]:
df1.head(5)

,user_id,device_category,total_session_duration
0,e3eabd43-23f3-4781-90ee-5a67e781e7e2,desktop,2019
1,e4ccc0ac-71b1-4d7c-b697-f9f76f1084d0,mobile,467
2,815ef036-f1ee-486e-85a2-facf11df24a1,mobile,540
3,c86127fb-bc84-488a-b961-2abc5213f6fd,mobile,161
4,1c101e61-f3cc-4d77-8803-e27aa80f2a59,desktop,81


In [123]:
fig = px.box(df1, x ='device_category', y = 'total_session_duration')
fig.show(renderer="colab")

#### Carrying out ANOVA testing to determine existence of correlation or not?

In [124]:
grps = pd.unique(df1.device_category)
print(grps)
data_groups = {grp:df1['total_session_duration'][df1['device_category']==grp] for grp in grps}

['desktop' 'mobile' 'tablet']


In [125]:
k = len(grps)                                # number of independent comaparison groups
print(k)
N = len(df1.values)                          # total number of samples
print(N)

# n = df1.groupby('device_category')['total_session_duration'].nunique()
n = dict(df1.groupby('device_category').size()) # number of samples in each group
print(n)

3
3764
{'desktop': 2255, 'mobile': 1430, 'tablet': 79}


In [126]:
from scipy import stats

F, p = stats.f_oneway(data_groups['desktop'], data_groups['mobile'], data_groups['tablet'])

F, p

(1.422647570398177, 0.24120460879072453)

Ans: Here, we have a p-value of 0.24 which is greater than 0.05 and 0.1 at 5 % and 10 % significance level. This means that the our sample statistics F isn't statistically significant to reject the nulle hypothesis and there is no correlation between screen size and total session duration both at 5% and 10 % significance level. 

## Q2. This data includes three partner sites: Freim.tv, News culturacolectiva.com, and culturacolcetiva.com. Based on Page_URL on which of the three partner sites do users have longer sessions?

In [127]:
df2 = users_data.filter(['user_id', 'traffic_source', 'total_session_duration'], axis=1)
print(df2.shape)

(3764, 3)


In [128]:
df2.head(5)

,user_id,traffic_source,total_session_duration
0,e3eabd43-23f3-4781-90ee-5a67e781e7e2,(direct),2019
1,e4ccc0ac-71b1-4d7c-b697-f9f76f1084d0,m.facebook.com,467
2,815ef036-f1ee-486e-85a2-facf11df24a1,google,540
3,c86127fb-bc84-488a-b961-2abc5213f6fd,m.facebook.com,161
4,1c101e61-f3cc-4d77-8803-e27aa80f2a59,tegger.io,81


In [129]:
df2['traffic_source'].value_counts()

tegger.io                                       1361
(direct)                                        1222
google                                           405
culturacolectiva.com                             251
news.culturacolectiva.com                        183
freim.tv                                         145
m.facebook.com                                    36
l.facebook.com                                    30
bing                                              21
t.co                                              20
culturacolectiva-com.cdn.ampproject.org           17
Cultura Colectiva                                 12
pinterest.com                                      8
hora724.com                                        8
taboola                                            6
blog.propsproject.com                              4
newsletter                                         3
lm.facebook.com                                    2
hu.pinterest.com                              

In [130]:
cat_friem = df2[df2['traffic_source'].isin(['freim.tv'])]
print(cat_friem.shape)
print(sum(cat_friem['total_session_duration']))

(145, 3)
12663667


In [131]:
cat_cult = df2[df2['traffic_source'].isin(['culturacolectiva.com','culturacolectiva-com.cdn.ampproject.org', 'Cultura Colectiva' ])]
print(cat_cult.shape)
print(sum(cat_cult['total_session_duration']))

(280, 3)
30335473


In [132]:
cat_news_cult = df2[df2['traffic_source'].isin(['news-culturacolectiva-com.cdn.ampproject.org','news.culturacolectiva.com'])]
print(cat_news_cult.shape)
print(sum(cat_news_cult['total_session_duration']))

(184, 3)
57543840


#### Ans: News.culturalcolective.com has the longer sessions among the three partner sites

## Q3. What combination of traffic source, browser language, browser, and mobile brand yields users with the longest sessions? What combination yields users with more page views?

In [134]:
# we already have traffic_source, browser, mobile brand, total session duration, total page views in user_data
# Let's create a new dataframe df3 with empyty column br_language which we need to fill from snowplow_data

df3 = users_data.filter(['user_id','traffic_source', 'browser','mobile_brand','total_session_duration', 'total_page_views'],axis=1)
df3['br_lang']= ""
print(df3.shape)

(3764, 7)


In [136]:
# for each user_id get the browser language from snowplot_data and put it into the br_lang column in df3
# It was confirmed that almost all the users browse the sites in a single language

for i in range(len(users_id)):
  temp_array = snowplow_data[snowplow_data['user_id']==users_id[i]]['br_lang'].value_counts().index.tolist()
  if temp_array:
    lang = temp_array[0]
  else:
    lang = ""
  df3.loc[df3['user_id'].isin([users_id[i]]), 'br_lang'] = str(lang)

df3['br_lang'].value_counts()

          2730
es-ES      544
en-US      353
de-DE       66
ko          38
es-mx       15
es-MX       13
es           2
es-AR        1
es-419       1
es-US        1
Name: br_lang, dtype: int64

For 70 % of the data in user_data, there was no br_lang found in snowplow_data. Hence, for such data an empty string was put in df3.

In [137]:
df3.head(5)

,user_id,traffic_source,browser,mobile_brand,total_session_duration,total_page_views,br_lang
0,e3eabd43-23f3-4781-90ee-5a67e781e7e2,(direct),Chrome,Google,2019,4,
1,e4ccc0ac-71b1-4d7c-b697-f9f76f1084d0,m.facebook.com,Chrome,Samsung,467,4,
2,815ef036-f1ee-486e-85a2-facf11df24a1,google,Chrome,Xiaomi,540,2,
3,c86127fb-bc84-488a-b961-2abc5213f6fd,m.facebook.com,Android Webview,Samsung,161,2,
4,1c101e61-f3cc-4d77-8803-e27aa80f2a59,tegger.io,Chrome,Google,81,3,en-US


In [139]:
traffic_cat = df3['traffic_source'].value_counts().index.to_list()
print("Num of Traffic Source: %d " % (len(traffic_cat)))
browser_cat = df3['browser'].value_counts().index.to_list()
print("Num of Browser: %d " % (len(browser_cat)))
mobile_cat = df3['mobile_brand'].value_counts().index.to_list()
print("Num of Mobile brands: %d" % (len(mobile_cat)))
browser_lang_cat = df3['br_lang'].value_counts().index.tolist()
print("Num of Browser Languages: %d" % (len(browser_lang_cat)))

Num of Traffic Source: 42 
Num of Browser: 11 
Num of Mobile brands: 31
Num of Browser Languages: 11


So there are 157542 possible combinations.

In [140]:
print(traffic_cat)
print(browser_cat)
print(mobile_cat)
print(browser_lang_cat)

['tegger.io', '(direct)', 'google', 'culturacolectiva.com', 'news.culturacolectiva.com', 'freim.tv', 'm.facebook.com', 'l.facebook.com', 'bing', 't.co', 'culturacolectiva-com.cdn.ampproject.org', 'Cultura Colectiva', 'pinterest.com', 'hora724.com', 'taboola', 'blog.propsproject.com', 'newsletter', 'lm.facebook.com', 'hu.pinterest.com', 'ampproject.org', 'news-staging.culturacolectiva.com', 'es.search.yahoo.com', 'news.google.com', 'mymoney777.icu', 'msn.com', 'pinterest.ca', 'ecosia.org', 'mx.search.yahoo.com', 'tejidosacrochetpasoapaso.com', 'nattittorres.wixsite.com', 'l.instagram.com', 'yahoo', 'propeller', 'pinterest.com.mx', 'aulavirtualinmaculadaloja.edu.ec', 'markets.businessinsider.com', 'ejemplos.co', 'id.pinterest.com', 'facebook.com', 'news-culturacolectiva-com.cdn.ampproject.org', 'transparentbusiness.com', 'duckduckgo']
['Chrome', 'Safari', 'Firefox', 'Edge', 'Opera', 'Samsung Internet', 'Android Webview', 'Puffin', 'YaBrowser', 'Safari (in-app)', 'UC Browser']
['Google', 

In [ ]:
rows_list = []
for i in traffic_cat:
  for j in browser_cat:
    for k in browser_lang_cat:
      for l in mobile_cat:
        result = df3[(df3['traffic_source']==i) & (df3['browser']==j) & (df3['br_lang']==k) & (df3['mobile_brand']==l)]
        if not result.empty:
          total_ses = sum(result['total_session_duration'])
          total_page = sum(result['total_page_views'])
          dict = {}
          dict.update({'traffic_source':i, 'browser': j, 'br_lang':k, 'mobile_brand': l ,'total_session_duration':total_ses, 'total_page_views':total_page})
          rows_list.append(dict)

df3_combination = pd.DataFrame(rows_list)


In [ ]:
print(df3_combination.shape)
df3_combination.head(10)

(217, 6)


,traffic_source,browser,br_lang,mobile_brand,total_session_duration,total_page_views
0,tegger.io,Chrome,,Google,44144569,385096
1,tegger.io,Chrome,,Apple,205306,13689
2,tegger.io,Chrome,,Samsung,62461943,23182122
3,tegger.io,Chrome,,Huawei,1070659,28858
4,tegger.io,Chrome,,Motorola,5555936,17122
5,tegger.io,Chrome,,Xiaomi,1390944,33515
6,tegger.io,Chrome,,LG,55404,328
7,tegger.io,Chrome,,OPPO,13524,90
8,tegger.io,Chrome,,ZTE,90970,603
9,tegger.io,Chrome,,Hisense,143441,1607


In [ ]:
df3_combination[df3_combination['total_session_duration']==df3_combination['total_session_duration'].max()]

,traffic_source,browser,br_lang,mobile_brand,total_session_duration,total_page_views
2,tegger.io,Chrome,,Samsung,62461943,23182122


Combination of traffic source, browser language, browser, and mobile brand that has longest total session time shown above.

In [ ]:
df3_combination[df3_combination['total_page_views']==df3_combination['total_page_views'].max()]

,traffic_source,browser,br_lang,mobile_brand,total_session_duration,total_page_views
2,tegger.io,Chrome,,Samsung,62461943,23182122


Combination of traffic source, browser language, browser, and mobile brand that has most page views shown above.